In [1]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle
from WavletTransform import wavlet_transform

#이번에도 실패하면 그냥 DNN시도해보자
base = 128  # 몇 일의 데이터를 쓸 것인가
period = 48   # 몇 step내를 예측할 것인가
step = 5  # 몇 step씩 이동할 것인가

#pickle_list=['3bull_1h_uptodate.pickle', 'etf_1h.pickle', 'stock_1h.pckle']
pickle_list=['Bitcoin_1h_uptodate.pickle', 'Bitcoin_1h.pickle']

for dict_file in pickle_list:
    X, Y = [],[]
    print(dict_file)
    with open(dict_file,"rb") as fr:
        container = pickle.load(fr)

    
    for name in tqdm(container):
        
        Data = container[name]
        Data.columns=["High", "Low", "Close", "Volume"]                
        Data["Volume"]=np.log(Data["Volume"] + 1)
        Data.dropna(subset=['Close'], inplace=True) 
#        print(Data)
        
              
            
        date_list = Data.iloc[base-1:].loc[:].index[:-period][::step]    # 예측하려는 날짜-1일까지 step씩
#        date_list = Data.iloc[base-1:].loc["202106050930":].index[:-period][::step]    # 예측하려는 날짜-1일까지 step씩
        #date_list = Data.iloc[base-1:].index[:-period]            #예측하려는 날짜-1
        
        # i는 예측하려는 날-1
        for i in date_list:
            
            
            close_std=Data.loc[:i].iloc[-base:]["Close"].std()    #수정종가의 표준편차
            Last_price = Data.loc[i]['Close'] + 0.0001
            y = Data.loc[i:].iloc[1:period + 1]["Close"]/Last_price - 1
            err=1
            
            for s in range(3, 2, -1):#3%는 올라야지
                fluct = y[abs(y) > 0.01*s]
                if np.shape(fluct)[0] > 0:
                    y=fluct.iloc[0]     #후에 어떤 추가적 변동이 있었든지 가장 먼저 변동한 것만 취급한다.
                    err=0
                    break
            if err==1:continue
            
            x = Data.loc[:i].iloc[-base:]           #예측일자 -base일
            if x.isnull().values.any() : continue   #null값 있으면 pass            
            x = (x-x.mean())/(x.std()+0.0001)        #정규화    
                        
            
            y = (np.sign(y)+1)//2  #올랐으면 1 내렸으면 0
            Y.append(y)
            X.append(x)    

    XX = np.array(X)
    YY = np.array(Y, dtype="uint8")

    I = np.arange(np.shape(YY)[0])
    np.random.shuffle(I)

    XX = XX[I]
    YY = YY[I]

    shape = np.shape(XX)
    print(shape)
    print(np.shape(YY[YY==0])), print(np.shape(YY[YY==1]))
    
    save_name=dict_file.split('.')[0]
    #균등화
    uI = YY==1
    dI = YY==0
    
    length = min(sum(uI), sum(dI))

    XX = np.concatenate( (XX[uI][:length], XX[dI][:length]  ), axis=0)
    YY = np.concatenate( (YY[uI][:length], YY[dI][:length]  ), axis=0)

    I = np.arange(np.shape(YY)[0])

    np.random.shuffle(I)

    XX = XX[I]
    YY = YY[I]
    
    np.save('X_%s_%d.npy' %(save_name, step), XX)
    np.save('Y_%s_%d.npy' %(save_name, step), YY)


    


  0%|                                                                                          | 0/126 [00:00<?, ?it/s]

Bitcoin_1h_uptodate.pickle


100%|████████████████████████████████████████████████████████████████████████████████| 126/126 [00:51<00:00,  2.45it/s]


(24387, 128, 4)
(13412,)
(10975,)


  0%|                                                                                          | 0/336 [00:00<?, ?it/s]

Bitcoin_1h.pickle


100%|████████████████████████████████████████████████████████████████████████████████| 336/336 [21:38<00:00,  3.86s/it]


(599536, 128, 4)
(331637,)
(267899,)
